In [14]:
# PROCEDURALLY GENERATED ITEMS
class Item:  
    RARITIES_VARIANTS = {
        "Common" : [ (1, 0) ],
        "Rare" : [ (2, 0), (1, 1) ],
        "Epic" : [ (2, 1), (1, 2) ],
        "Legendary" : [ (2, 2), (3, 1), (1, 3) ]       
    }
    
    TYPES = {
        "Headgear" : {
            "stats+" : ["DEF", "CST"],
            "stats-" : ["STR", "PER"],
            "bonuses+" : ["%_ARMOR", "ARMOR", "%HP", "HP"],
            "bonuses-" : ["%_ATK_PWR", "ATK_PWR", "CRIT_DMG", "CRIT_CHANCE"]            
        },
        "Armor" : {
            "stats+" : ["DEF", "CST"],
            "stats-" : ["STR", "PER"],
            "bonuses+" : ["%_ARMOR", "ARMOR", "%HP", "HP"],
            "bonuses-" : ["%_ATK_PWR", "ATK_PWR", "CRIT_DMG", "CRIT_CHANCE"]            
        },
        "Weapon" : {
            "stats+" : ["STR", "AGI"],
            "stats-" : ["DEF", "CST"],
            "bonuses+" : ["%_ATK_PWR", "ATK_PWR", "CRIT_DMG", "CRIT_CHANCE"],
            "bonuses-" : ["%_ARMOR", "ARMOR", "%HP", "HP"]
        },
        "Charm" : {
            "stats+" : ["LCK", "PER"],
            "stats-" : ["AGI", "CST"],
            "bonuses+" : ["%_ATK_PWR", "ATK_PWR", "CRIT_DMG", "CRIT_CHANCE"],
            "bonuses-" : ["%_ARMOR", "ARMOR", "%HP", "HP"]            
        }
    }
       
    def __init__(self, luck : int, level : int, seed : int = None, item_type : str = None):
        
        import numpy.random as rnd
        import numpy as np
        from scipy.special import softmax
 
        self.stats = {
            "STR" : 0,
            "AGI" : 0,
            "PER" : 0,
            "DEF" : 0,
            "LCK" : 0,
            "CST" : 0
        }
        
        # TODO: POSORTOWAĆ ŻEBY % BYŁY NA KOŃCU
        self.bonuses = {
            "%_ATK_PWR" : 1.0,
            "%_ARMOR" : 1.0,
            "%_HP" : 1.0,
            "ATK_PWR" : 0,
            "ARMOR" : 0,
            "HP" : 0,
            "CRIT_DMG" : 0,
            "CRIT_CHANCE" : 0,
            "EVA_CHANCE" : 0
            # TODO: dopisać wiemcej :-DDD
        }

        self.rarity = None
        self.item_type = None

        if seed:
            rnd.seed(seed)
            
        # Get item_type:
        if not item_type:
            self.item_type = rnd.choice(list(self.TYPES.keys()))
        else:
            self.item_type = item_type
        # Get rarity:
        legendary_chance = 0.05 + luck / 100
        epic_chance = 0.10 + luck / 100
        rare_chance = 0.25 + luck / 100
        common_chance = max(0.0, 1.0 - legendary_chance - epic_chance - rare_chance)
        rarity_chances = np.array([common_chance, rare_chance, epic_chance, legendary_chance])
        rarity_chances /= rarity_chances.sum()
        self.rarity = rnd.choice(
            ["Common", "Rare", "Epic", "Legendary"],
            p=rarity_chances,
            size = 1
        )[0]
        
        # Get variant of item according to rarity:
        variant = self.RARITIES_VARIANTS[self.rarity][rnd.choice(len(self.RARITIES_VARIANTS[self.rarity]))]
        stats_to_gen, bonuses_to_gen = variant
        
        # Generate stats according to variant
        stats_chances = []
        base_chance = 1/len(self.stats)
        for key in self.stats:
            if key in self.TYPES[self.item_type]["stats+"]:
                stats_chances.append(base_chance * 1.5)
                continue
            if key in self.TYPES[self.item_type]["stats-"]:
                stats_chances.append(base_chance * 0.67)
                continue
            stats_chances.append(base_chance)
        
        stats_chances = np.array(stats_chances)
        stats_chances /= stats_chances.sum()
        
        stats_to_gen = list(rnd.choice(list(self.stats.keys()), size = stats_to_gen, replace = False, p = stats_chances))
        
        for stat in stats_to_gen:
            self.stats[stat] += max(1, round(rnd.normal(loc = level * 3 / 10, scale = 2.2)))
            
        # Generate bonuses according to variant
        if bonuses_to_gen:
            bonuses_chances = []
            base_chance = 1/len(self.bonuses)
            for key in self.bonuses:
                if key in self.TYPES[self.item_type]["bonuses+"]:
                    bonuses_chances.append(base_chance * 1.5)
                    continue
                if key in self.TYPES[self.item_type]["bonuses-"]:
                    bonuses_chances.append(base_chance * 0.67)
                    continue
                bonuses_chances.append(base_chance)
        
            bonuses_chances = np.array(bonuses_chances)
            bonuses_chances /= bonuses_chances.sum()
            
            bonuses_to_gen = list(rnd.choice(list(self.bonuses.keys()), size = bonuses_to_gen, replace = False, p = bonuses_chances))
        
            for bonus in bonuses_to_gen:
                if "%" in bonus:
                    self.bonuses[bonus] += max(10, round(rnd.normal(loc = level*3, scale = level/3)))/100
                    continue
                self.bonuses[bonus] += max(10, round(rnd.normal(loc = level, scale = 5)))
            
    def to_dict(self):
        return {
            "item_type" : self.item_type,
            "rarity" : self.rarity,
            "stats" : { k:v for k, v in self.stats.items() if v != 0},
            "bonuses" : { k:v for k, v in self.bonuses.items() if ("%" in k and v != 1.0) or (not "%" in k and v != 0) }
        }

In [20]:
EXP_FOR_LEVEL = { 100 + 20}

class Character:

    def __init__(self):
        
        self.level = 1
        self.xp = 0

        # MACRO STATS
        self.strength = 0
        self.agility = 0
        self.perception = 0
        self.defense = 0
        self.luck = 0
        self.constitution = 0
        
        # DERIVED STATS
        self.attack_power = 20.0
        self.armor = 10.0
        self.health = 50.0
        self.crit_dmg = 1.3
        self.crit_chance = 0.05
        self.eva_chance = 0.03
        self.accuracy = 0.9
        self.rev_chance = 0.01
        self.lifesteal = 0.0

        self.items = {
            "Headgear" : None,
            "Armor" : None,
            "Weapon" : None,
            "Charm" : None
        }
        
    def generate_basic_enemy(self, level):
        pass

    def increase_stats(self, strength=0, agility=0, perception=0, defense=0, luck=0, constitution=0):
        self.strength += strength
        self.agility += agility
        self.perception += perception
        self.defense += defense
        self.luck += luck
        self.constitution += constitution
        
    def get_stats(self):
        return {
            "STR" : self.strength,
            "AGI" : self.agility,
            "PER" : self.perception,
            "DEF" : self.defense,
            "LCK" : self.luck,
            "CST" : self.constitution
        }
    
    def get_stats_with_items(self):
        cur_stats = self.get_stats()
        
        # Add stats from items
        for item in self.items.values():
            if item == None:
                continue
            for stat, value in item.stats.items():
                cur_stats[stat] += value
        return cur_stats
    
    def get_derived_stats(self):
        cur_stats = self.get_stats_with_items()
        derived_stats = {
            "ATK_PWR" : self.attack_power + 10.0 * cur_stats["STR"],
            "ARMOR" : self.armor + 5.0 * cur_stats["DEF"],
            "HP" : self.health + 25.0 * cur_stats["CST"],
            "CRIT_DMG" : self.crit_dmg + 0.05 * cur_stats["STR"],
            "CRIT_CHANCE" : self.crit_chance + 0.025 * cur_stats["LCK"],
            "EVA_CHANCE" : self.eva_chance + 0.015 * cur_stats["AGI"],
            "ACCURACY" : self.accuracy + 0.01 * cur_stats["AGI"],
            "REV_CHANCE" : self.rev_chance + 0.1 * cur_stats["PER"],
            "LIFESTEAL" : self.lifesteal + 0.025 * cur_stats["CST"]
        }
        
        percentage_bonuses = {}
        for item in self.items.values():
            if item == None:
                continue
            for bonus, value in item.bonuses.items():
                if "%" in bonus:
                    bonus_name = bonus[2:]
                    if bonus_name in percentage_bonuses:
                        percentage_bonuses[bonus_name] *= value
                    else:
                        percentage_bonuses[bonus_name] = value
                    continue
                derived_stats[bonus] += value
            for bonus, value in percentage_bonuses.items():
                derived_stats[bonus] *= value
        
        return derived_stats
                
    def _add_random_items(self):
        for key in self.items:
            self.items[key] = Item(self.luck, self.level, item_type=key)

In [21]:
player = Character()

In [22]:
player.get_stats()

{'STR': 0, 'AGI': 0, 'PER': 0, 'DEF': 0, 'LCK': 0, 'CST': 0}

In [23]:
player.get_derived_stats()

{'ATK_PWR': 20.0,
 'ARMOR': 10.0,
 'HP': 50.0,
 'CRIT_DMG': 1.3,
 'CRIT_CHANCE': 0.05,
 'EVA_CHANCE': 0.03,
 'ACCURACY': 0.9,
 'REV_CHANCE': 0.01,
 'LIFESTEAL': 0.0}

In [24]:
player._add_random_items()

In [25]:
player.get_derived_stats()

{'ATK_PWR': 50.0,
 'ARMOR': 25.0,
 'HP': 90.75000000000001,
 'CRIT_DMG': 1.4500000000000002,
 'CRIT_CHANCE': 0.07500000000000001,
 'EVA_CHANCE': 0.03,
 'ACCURACY': 0.9,
 'REV_CHANCE': 0.01,
 'LIFESTEAL': 0.025}

In [155]:
Item(luck=10, level=10).to_dict()

{'item_type': 'Armor',
 'rarity': 'Epic',
 'stats': {'STR': 5, 'AGI': 3},
 'bonuses': {'%_ARMOR': 1.27}}

In [156]:
sorted(["%", "A"])

['%', 'A']

In [ ]:
from enum import Enum
